# Coursera capstone project 

### The goal of this project is to find an ideal place to open a new Coffee place in the city of Casablanca.
### In order to do so , we will scrape most of the neighborhoods Names of the wikipedia page : https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Casablanca , after that  we will get the goegraphical data using the foursquare api . Then , we'll do some analysis on the data , and at the end , we will make some maps using folium library to help us find a good place for the New coffe place .

### Importing Libraries.

In [1]:
import numpy as np 

import pandas as pd 
from pandas.io.json import json_normalize 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from requests import get

import folium 
from folium import plugins
from folium.plugins import HeatMap

import bs4
from bs4 import BeautifulSoup 


print('Libraries imported.')

Libraries imported.


### Getting and Cleaning the Data.

In [2]:
url = 'https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Casablanca'
response = get(url)
print(response.text[:1000])


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Category:Neighbourhoods of Casablanca - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"c170b555-9f55-4608-be8f-e3c1c58f092c","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Neighbourhoods_of_Casablanca","wgTitle":"Neighbourhoods of Casablanca","wgCurRevisionId":695035595,"wgRevisionId":695035595,"wgArticleId":33232035,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Subdivisions of Casablanca","Suburbs by city"],"wgPageContentLanguage":"en","wgPageContentModel

In [3]:
### We will use the library bs4 to scrape the data

In [4]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)
bs4.BeautifulSoup

bs4.BeautifulSoup

In [5]:
Data = html_soup.find_all('div', class_ = 'mw-category')
print(type(Data))
print(len(Data))
Data=Data[0]

<class 'bs4.element.ResultSet'>
1


#### The infromation we want is in the "div" sections .

In [6]:
Data=Data.findAll('div')
Data

[<div class="mw-category-group"><h3>A</h3>
 <ul><li><a href="/wiki/Ain_Diab" title="Ain Diab">Ain Diab</a></li>
 <li><a href="/wiki/A%C3%AFn_Seba%C3%A2" title="Aïn Sebaâ">Aïn Sebaâ</a></li>
 <li><a href="/wiki/Anfa" title="Anfa">Anfa</a></li></ul></div>,
 <div class="mw-category-group"><h3>B</h3>
 <ul><li><a href="/wiki/Belvedere_(Casablanca)" title="Belvedere (Casablanca)">Belvedere (Casablanca)</a></li>
 <li><a href="/wiki/Bourgogne_(Casablanca)" title="Bourgogne (Casablanca)">Bourgogne (Casablanca)</a></li></ul></div>,
 <div class="mw-category-group"><h3>D</h3>
 <ul><li><a href="/wiki/Derb_Ghallef" title="Derb Ghallef">Derb Ghallef</a></li>
 <li><a href="/wiki/Derb_Sultan" title="Derb Sultan">Derb Sultan</a></li></ul></div>,
 <div class="mw-category-group"><h3>H</h3>
 <ul><li><a href="/wiki/Hay_El_Hanaa" title="Hay El Hanaa">Hay El Hanaa</a></li>
 <li><a href="/wiki/Hay_El_Hassani" title="Hay El Hassani">Hay El Hassani</a></li>
 <li><a href="/wiki/Hay_Salama" title="Hay Salama">Hay 

#### Creating a loop to get the names of neihborhoods , and adding some missing neihborhoods manually.

In [7]:
Neighborhoods_list=[]
for k in range(0,11):
    for s in Data[k].ul.text.split('\n'):
        s=s+",Casa"
        Neighborhoods_list.append(s)

Neighborhoods_list[21]='Sidi Maârouf,Casablanca'      
Neighborhoods_list[14]='Maârif,Casablanca'
Neighborhoods_list[15]='LOasis,Casa'
Neighborhoods_list[7]='Hay El Hana,Casa'
Neighborhoods_list[0] ='Ain Diab,Casablanca'
Neighborhoods_list[9]='Hay Salama,Casablanca'        
Neighborhoods_list[18]='Roches Noires,Casa'
Neighborhoods_list[10]="Habbous,Casa"   
Neighborhoods_list[13]="Lamkansa,Casa"  
Neighborhoods_list.pop()
Neighborhoods_list.append('sidi moumen casablanca')
Neighborhoods_list.append('Sidi Bernoussi casablanca')
Neighborhoods_list.append('hay mohammadi casablanca')
Neighborhoods_list.append('ben msik casablanca')
Neighborhoods_list.append('ain chock casablanca')
Neighborhoods_list.append('Sidi Othmane casablanca')
Neighborhoods_list.append('Al Fida casablanca')
Neighborhoods_list.append('Sidi Belyout casablanca')
Neighborhoods_list.append('moulay rachid casablanca')
Neighborhoods_list.append("mechouar casablanca")

Neighborhoods_list

['Ain Diab,Casablanca',
 'Aïn Sebaâ,Casa',
 'Anfa,Casa',
 'Belvedere (Casablanca),Casa',
 'Bourgogne (Casablanca),Casa',
 'Derb Ghallef,Casa',
 'Derb Sultan,Casa',
 'Hay El Hana,Casa',
 'Hay El Hassani,Casa',
 'Hay Salama,Casablanca',
 'Habbous,Casa',
 'Inara (Casablanca),Casa',
 'La Colline (Casablanca),Casa',
 'Lamkansa,Casa',
 'Maârif,Casablanca',
 'LOasis,Casa',
 'Oulfa,Casa',
 'Racine (Casablanca),Casa',
 'Roches Noires,Casa',
 'Salmia 2 (Casablanca),Casa',
 'Sbata,Casa',
 'Sidi Maârouf,Casablanca',
 'sidi moumen casablanca',
 'Sidi Bernoussi casablanca',
 'hay mohammadi casablanca',
 'ben msik casablanca',
 'ain chock casablanca',
 'Sidi Othmane casablanca',
 'Al Fida casablanca',
 'Sidi Belyout casablanca',
 'moulay rachid casablanca',
 'mechouar casablanca']

In [8]:
#creating the dataframe
Neighborhoods=pd.DataFrame(Neighborhoods_list,columns=['Neighborhood']) 
Neighborhoods.head(10)

,Neighborhood
0,"Ain Diab,Casablanca"
1,"Aïn Sebaâ,Casa"
2,"Anfa,Casa"
3,"Belvedere (Casablanca),Casa"
4,"Bourgogne (Casablanca),Casa"
5,"Derb Ghallef,Casa"
6,"Derb Sultan,Casa"
7,"Hay El Hana,Casa"
8,"Hay El Hassani,Casa"
9,"Hay Salama,Casablanca"


In [9]:
# loop for observing the goegraphical data 
for k in Neighborhoods['Neighborhood']:
    address = k
    geolocator = Nominatim(user_agent="casa_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(k,latitude, longitude))

The geograpical coordinate of Ain Diab,Casablanca are 33.5922876, -7.6764187.
The geograpical coordinate of Aïn Sebaâ,Casa are 33.6085906, -7.524144079373121.
The geograpical coordinate of Anfa,Casa are 33.5926317, -7.6721766.
The geograpical coordinate of Belvedere (Casablanca),Casa are 33.591292550000006, -7.590353351181981.
The geograpical coordinate of Bourgogne (Casablanca),Casa are 33.5978925, -7.6381616.
The geograpical coordinate of Derb Ghallef,Casa are 33.57061365, -7.6300569530492695.
The geograpical coordinate of Derb Sultan,Casa are 33.5721699, -7.5958496.
The geograpical coordinate of Hay El Hana,Casa are 33.57707895, -7.666133176827215.
The geograpical coordinate of Hay El Hassani,Casa are 33.5670464, -7.6784886.
The geograpical coordinate of Hay Salama,Casablanca are 33.5605055, -7.5581006.
The geograpical coordinate of Habbous,Casa are 33.5751052, -7.6448335.
The geograpical coordinate of Inara (Casablanca),Casa are 33.537, -7.6.
The geograpical coordinate of La Collin

In [10]:
# loop for creating  latitude and longitude lists 
Latitude=[]
Longitude=[]
for k in Neighborhoods['Neighborhood']:
    address = k
    geolocator = Nominatim(user_agent="casa_explorer")
    location = geolocator.geocode(address)
    Latitude.append(location.latitude)
    Longitude.append(location.longitude)

#correcting wrong data
Latitude[13]=33.53372
Longitude[13]=-7.57393
Latitude[25]=33.556335
Longitude[25]=-7.581414

#adding the lists to the dataframe
Neighborhoods['Latitude']=Latitude
Neighborhoods['Longitude']=Longitude
Neighborhoods.head(10)

,Neighborhood,Latitude,Longitude
0,"Ain Diab,Casablanca",33.592288,-7.676419
1,"Aïn Sebaâ,Casa",33.608591,-7.524144
2,"Anfa,Casa",33.592632,-7.672177
3,"Belvedere (Casablanca),Casa",33.591293,-7.590353
4,"Bourgogne (Casablanca),Casa",33.597893,-7.638162
5,"Derb Ghallef,Casa",33.570614,-7.630057
6,"Derb Sultan,Casa",33.572170,-7.595850
7,"Hay El Hana,Casa",33.577079,-7.666133
8,"Hay El Hassani,Casa",33.567046,-7.678489
9,"Hay Salama,Casablanca",33.560505,-7.558101


### Let's draw our first map and visualize the neihborhoods on it .

In [11]:
address = 'Casablanca'

geolocator = Nominatim(user_agent="Casablanca")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Casablanca are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Casablanca are 33.5950627, -7.6187768.


In [12]:
# create map of Casablanca using latitude and longitude values
map_Casa = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng,neighborhood in zip(Neighborhoods['Latitude'], Neighborhoods['Longitude'], Neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Casa)  
    
map_Casa

### let's work on getting the venues now.

In [13]:
#Defining Foursquare Credentials and Version
LIMIT = 1000 
radius = 80000 
CLIENT_ID = 'A4QK2HRDTNWL1AQ2S4QOORSXJZYFYTPWON411PWNCFUJW5PA' 
CLIENT_SECRET = 'HG3UHJXOLWKUN0TMS504V5W3CSCWAEJAXYNXVNBC0FQANW20' 
VERSION = '20180605' 

print('The credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

The credentails:
CLIENT_ID: A4QK2HRDTNWL1AQ2S4QOORSXJZYFYTPWON411PWNCFUJW5PA
CLIENT_SECRET:HG3UHJXOLWKUN0TMS504V5W3CSCWAEJAXYNXVNBC0FQANW20


In [14]:
#function to get the nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [15]:
#getting near by venues into a data frame 
NearbyVenues = getNearbyVenues(names=Neighborhoods['Neighborhood'],
                                   latitudes=Neighborhoods['Latitude'],
                                   longitudes=Neighborhoods['Longitude']
                                  )


Ain Diab,Casablanca
Aïn Sebaâ,Casa
Anfa,Casa
Belvedere (Casablanca),Casa
Bourgogne (Casablanca),Casa
Derb Ghallef,Casa
Derb Sultan,Casa
Hay El Hana,Casa
Hay El Hassani,Casa
Hay Salama,Casablanca
Habbous,Casa
Inara (Casablanca),Casa
La Colline (Casablanca),Casa
Lamkansa,Casa
Maârif,Casablanca
LOasis,Casa
Oulfa,Casa
Racine (Casablanca),Casa
Roches Noires,Casa
Salmia 2 (Casablanca),Casa
Sbata,Casa
Sidi Maârouf,Casablanca
sidi moumen casablanca
Sidi Bernoussi casablanca
hay mohammadi casablanca
ben msik casablanca
ain chock casablanca
Sidi Othmane casablanca
Al Fida casablanca
Sidi Belyout casablanca
moulay rachid casablanca
mechouar casablanca


In [16]:
NearbyVenues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Ain Diab,Casablanca",33.592288,-7.676419,Basmane Restaurant,33.591577,-7.677553,Moroccan Restaurant
1,"Ain Diab,Casablanca",33.592288,-7.676419,Le Boudoir,33.593305,-7.677625,Lounge
2,"Ain Diab,Casablanca",33.592288,-7.676419,Sinatra,33.595661,-7.677022,Italian Restaurant
3,"Ain Diab,Casablanca",33.592288,-7.676419,Sun Beach - CCC,33.595406,-7.678437,Pool
4,"Ain Diab,Casablanca",33.592288,-7.676419,Azur Hotel Casablanca,33.595360,-7.676824,Hotel
5,"Ain Diab,Casablanca",33.592288,-7.676419,Hotel Val D'Anfa,33.591664,-7.678328,Hotel
6,"Ain Diab,Casablanca",33.592288,-7.676419,Hotel Club Val d'Anfa,33.591684,-7.678351,Hotel
7,"Ain Diab,Casablanca",33.592288,-7.676419,Amstrong Legend,33.595402,-7.677080,Jazz Club
8,"Ain Diab,Casablanca",33.592288,-7.676419,Le Squat,33.593145,-7.677817,Nightclub
9,"Ain Diab,Casablanca",33.592288,-7.676419,Mercat,33.596119,-7.678204,Tapas Restaurant


In [17]:
# count how many venues in each neighborhood
NearbyVenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Ain Diab,Casablanca",41,41,41,41,41,41
Al Fida casablanca,6,6,6,6,6,6
"Anfa,Casa",19,19,19,19,19,19
"Aïn Sebaâ,Casa",4,4,4,4,4,4
"Belvedere (Casablanca),Casa",4,4,4,4,4,4
"Bourgogne (Casablanca),Casa",6,6,6,6,6,6
"Derb Ghallef,Casa",7,7,7,7,7,7
"Derb Sultan,Casa",5,5,5,5,5,5
"Habbous,Casa",9,9,9,9,9,9


In [18]:
# how many venue peer category
NearbyVenues['Venue Category'].value_counts()

Café                             52
Hotel                            19
Fast Food Restaurant             16
Italian Restaurant               15
Coffee Shop                      10
Pizza Place                       8
Diner                             8
Lounge                            6
Restaurant                        6
Bakery                            6
Shopping Mall                     5
Sandwich Place                    5
Nightclub                         5
Pub                               5
Moroccan Restaurant               5
French Restaurant                 5
Sushi Restaurant                  5
Middle Eastern Restaurant         4
Tram Station                      4
Pool Hall                         3
Juice Bar                         3
Gym                               3
Burger Joint                      3
Japanese Restaurant               3
Ice Cream Shop                    3
Department Store                  3
Flea Market                       3
Breakfast Spot              

#### We are interested in Café and Coffe Shop categories , from above we can see that Coffe places are dominating the venues.

In [19]:
print('There are {} uniques categories.'.format(len(NearbyVenues['Venue Category'].unique())))

There are 85 uniques categories.


In [20]:
# one hot encoding
Casa_onehot = pd.get_dummies(NearbyVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Casa_onehot['Neighborhood'] = NearbyVenues['Neighborhood'] 

# move neighborhood column to the first column
col_name="Neighborhood"
first_col =Casa_onehot.pop(col_name)
Casa_onehot.insert(0, col_name, first_col)
Casa_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Astrologer,BBQ Joint,Bakery,Beach,Big Box Store,Bookstore,Botanical Garden,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cupcake Shop,Department Store,Diner,Doner Restaurant,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Court,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,Grocery Store,Gym,Harbor / Marina,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Moroccan Restaurant,Night Market,Nightclub,Noodle House,Park,Pharmacy,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Pub,Resort,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Spa,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Theater,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,"Ain Diab,Casablanca",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Ain Diab,Casablanca",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Ain Diab,Casablanca",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Ain Diab,Casablanca",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Ain Diab,Casablanca",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
Casa_onehot.shape

(287, 85)

In [22]:
#grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Casa_grouped= Casa_onehot.groupby('Neighborhood').mean().reset_index()
Casa_grouped

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Astrologer,BBQ Joint,Bakery,Beach,Big Box Store,Bookstore,Botanical Garden,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cupcake Shop,Department Store,Diner,Doner Restaurant,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food Court,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,Grocery Store,Gym,Harbor / Marina,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Moroccan Restaurant,Night Market,Nightclub,Noodle House,Park,Pharmacy,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Pub,Resort,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Spa,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Theater,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,"Ain Diab,Casablanca",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.170732,0.0,0.000000,0.02439,0.000000,0.00000,0.00000,0.000000,0.024390,0.02439,0.0,0.000,0.097561,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02439,0.000000,0.000000,0.000000,0.146341,0.000000,0.000000,0.048780,0.000000,0.02439,0.000000,0.00000,0.073171,0.02439,0.000000,0.0,0.048780,0.0,0.073171,0.02439,0.0,0.0,0.000000,0.0,0.02439,0.000000,0.000000,0.024390,0.02439,0.073171,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000,0.02439,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
1,Al Fida casablanca,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.166667,0.000000,0.00000,0.0,0.000,0.333333,0.0,0.166667,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.166667,0.00000,0.0,0.00000,0.166667,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
2,"Anfa,Casa",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.0,0.00,0.000000,0.210526,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.052632,0.00000,0.0,0.000,0.052632,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.052632,0.052632,0.000000,0.052632,0.105263,0.000000,0.00000,0.000000,0.00000,0.105263,0.00000,0.052632,0.0,0.000000,0.0,0.105263,0.00000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.052632,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.052632
3,"Aïn Sebaâ,Casa",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.250000,0.0,0.000000,0.00000,0.250000,0.00000,0.00000,0.000000,0.250000,0.00000,0.0,0.000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
4,"Belvedere (Casablanca),Casa",0.0,0.000000,0.000000,0.000000,0.0000

In [23]:
Casa_grouped.shape

(31, 85)

### Let's check the top 5 dominating categories in each Neighborhood.

In [24]:
num_top_venues = 5

for hood in Casa_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Casa_grouped[Casa_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ain Diab,Casablanca----
                  venue  freq
0                  Café  0.17
1                 Hotel  0.15
2  Fast Food Restaurant  0.10
3                Lounge  0.07
4             Nightclub  0.07


----Al Fida casablanca----
                  venue  freq
0  Fast Food Restaurant  0.33
1           Flea Market  0.17
2      Department Store  0.17
3          Soccer Field  0.17
4           Supermarket  0.17


----Anfa,Casa----
                venue  freq
0                Café  0.21
1  Italian Restaurant  0.11
2           Nightclub  0.11
3              Lounge  0.11
4                 Pub  0.05


----Aïn Sebaâ,Casa----
                venue  freq
0  Italian Restaurant  0.25
1                Café  0.25
2               Diner  0.25
3         Coffee Shop  0.25
4                 Pub  0.00


----Belvedere (Casablanca),Casa----
                  venue  freq
0  Fast Food Restaurant  0.25
1                 Hotel  0.25
2          Tram Station  0.25
3           Pizza Place  0.25
4     Accessor

In [25]:
#function to return the common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Let's create a dataframe of the top 10 venues for each neighborhood.

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Casa_grouped['Neighborhood']

for ind in np.arange(Casa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Casa_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Ain Diab,Casablanca",Café,Hotel,Fast Food Restaurant,Restaurant,Nightclub,Lounge,Moroccan Restaurant,Italian Restaurant,Mediterranean Restaurant,Cocktail Bar
1,Al Fida casablanca,Fast Food Restaurant,Department Store,Flea Market,Supermarket,Soccer Field,French Restaurant,Doner Restaurant,Eastern European Restaurant,Farmers Market,Fish & Chips Shop
2,"Anfa,Casa",Café,Italian Restaurant,Lounge,Nightclub,Yoga Studio,Pub,Beach,Diner,Fast Food Restaurant,Hookah Bar
3,"Aïn Sebaâ,Casa",Diner,Café,Coffee Shop,Italian Restaurant,Astrologer,Gastropub,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food
4,"Belvedere (Casablanca),Casa",Hotel,Tram Station,Pizza Place,Fast Food Restaurant,French Restaurant,Eastern European Restaurant,Farmers Market,Fish & Chips Shop,Flea Market,Food
5,"Bourgogne (Casablanca),Casa",Middle Eastern Restaurant,Café,Pizza Place,Bakery,Shopping Mall,Yoga Studio,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
6,"Derb Ghallef,Casa",Print Shop,Tram Station,Train Station,Gym,Coffee Shop,Flea Market,Café,Yoga Studio,Food,Eastern European Restaurant
7,"Derb Sultan,Casa",Fast Food Restaurant,Pharmacy,Juice Bar,Snack Place,Café,Yoga Studio,Food Court,Farmers Market,Fish & Chips Shop,Flea Market
8,"Habbous,Casa",Café,Snack Place,Japanese Restaurant,Grocery Store,Sushi Restaurant,Sandwich Place,Yoga Studio,Eastern European Restaurant,Farmers Market,Fast Food Restaurant
9,"Hay El Hana,Casa",Accessories Store,Miscellaneous Shop,Chinese Restaurant,Night Market,Sports Club,Gym,Food Court,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


## Let's select only the coffe places from the venues list for our final analysis.

In [27]:
cafe_places=NearbyVenues[NearbyVenues['Venue Category'].isin(('Café','Coffee Shop'))].reset_index(drop=True)
cafe_places.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Ain Diab,Casablanca",33.592288,-7.676419,Le Crépuscule,33.593492,-7.679094,Café
1,"Ain Diab,Casablanca",33.592288,-7.676419,Gossip Beach,33.594370,-7.678767,Café
2,"Ain Diab,Casablanca",33.592288,-7.676419,Cafe Calypso,33.593473,-7.678322,Café
3,"Ain Diab,Casablanca",33.592288,-7.676419,Hooka Blanca,33.591470,-7.678073,Café
4,"Ain Diab,Casablanca",33.592288,-7.676419,Tropicana,33.596276,-7.676693,Café
5,"Ain Diab,Casablanca",33.592288,-7.676419,XS Lounge Casablanca,33.594576,-7.675280,Café
6,"Ain Diab,Casablanca",33.592288,-7.676419,Tropicana Terrasse,33.596304,-7.676723,Café
7,"Aïn Sebaâ,Casa",33.608591,-7.524144,CAFE ADANA,33.608835,-7.520845,Café
8,"Aïn Sebaâ,Casa",33.608591,-7.524144,Half Moon,33.607218,-7.524362,Coffee Shop
9,"Anfa,Casa",33.592632,-7.672177,Roosevelt Café,33.594718,-7.667885,Café


### Let's create a map with the venues on it .

In [28]:
cafe_map = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng,venue in zip(cafe_places['Venue Latitude'], cafe_places['Venue Longitude'],cafe_places['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cafe_map) 
cafe_map

### There are Many Coffe places all over the city ! let's create a heat map to make it easier for us to choose a good location for the new Coffe shop.

In [29]:
cafe_heat_map = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng,venue in zip(cafe_places['Venue Latitude'], cafe_places['Venue Longitude'],cafe_places['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cafe_heat_map) 
cafe_heat_map



HeatMap(data=cafe_places[['Venue Latitude', 'Venue Longitude']].groupby(['Venue Latitude', 'Venue Longitude']).sum().reset_index().values.tolist(), radius=25, max_zoom=13).add_to(cafe_heat_map)
cafe_heat_map

# Conclusion :

### Ain-Chok,Boulevard Panoramique is one of the possible Locations in Casablanca to open a new Coffe place because it's far away from the heated areas ,somehow in the middle of the city ,and at the same time it is close to the main highway  !

### Let's visualize the location we choose for our new coffe place on the map, along with the others.

In [30]:

New_cafe_heat_map = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng,venue in zip(cafe_places['Venue Latitude'], cafe_places['Venue Longitude'],cafe_places['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(New_cafe_heat_map) 
    folium.CircleMarker(
        [33.554568, -7.607658],
        radius=10,
        popup="The New Cafe!",
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
    parse_html=False).add_to(New_cafe_heat_map)

HeatMap(data=cafe_places[['Venue Latitude', 'Venue Longitude']].groupby(['Venue Latitude', 'Venue Longitude']).sum().reset_index().values.tolist(), radius=25, max_zoom=13).add_to(New_cafe_heat_map)
New_cafe_heat_map